In [1]:
import os, sys
sys.path.append(os.path.abspath('../..'))

import data.cyberkg_IBM.pkg.sysflow as sf
import data.cyberkg_IBM.pkg.cwe_miti as cm

567it [00:23, 24.52it/s]
166596it [01:05, 2536.91it/s]
100%|████████████████████████████████████████████████████████████████████████████████████████████████| 567/567 [01:27<00:00,  6.45it/s]


In [2]:
tech = 'T1134' # MRR, HIT@K, pre/recall
thre = 100
n_cwe = 20
thre_cves, thre_scores = sf.ttp_cve_link(n_cve = thre, tech = tech, multiview=False, verbose=False)
cwe_sort, score_sort = sf.cwe_cve_link(thre_cves, thre_scores) # sorted
# for i in range(len(cwe_sort)):
#     print(cwe_sort[i], ' \t', score_sort[i])
    
cwe_miti_curtech = []

cwe_phase_st_dict = cm.sum_miti(save=True)
for i in range(min(len(cwe_sort), n_cwe)):
    cwe, phase = cwe_sort[i], 'Operation'
    cwe = cwe.split(':')[-1]
#     print('CWE - %s, Phase - %s' % (str(cwe), phase))

    for st, txt_list in cwe_phase_st_dict[str(cwe)][phase].items():
        idx = 0
        for txt in txt_list:
            first_s = txt.split('. ')[0] + '.'
            if first_s.startswith("Effectiveness:"):
                continue
            idx += 1
#             print(st, '|', '(%d)' % idx, first_s)
            if first_s not in cwe_miti_curtech:
                cwe_miti_curtech.append(st)
                cwe_miti_curtech.append(first_s)
cwe_miti_curtech = '\n'.join(cwe_miti_curtech)
cwe_miti_curtech
# cwe_miti_curtech.split('\n')[1::2]

'Firewall\nUse an application firewall that can detect attacks against this weakness.\nEnvironment Hardening\nWhen using PHP, configure the application so that it does not use register_globals.\nNo Strategy\nVery carefully manage the setting, management, and handling of privileges.\nNo Strategy\nProtect log files against unauthorized read/write..'

### CWE - MITI - DEF

In [3]:
import json
import numpy as np
from collections import defaultdict
from data.cyberkg_IBM.pkg.sim_mitre import cwe_mitre_miti_cossim

mitre_miti = json.load(open('/home/zxx5113/IBM/data/cyberkg_IBM/save/mitre-attack/mitigations.json', 'r'))

miti_codes, miti_names, miti_descs = [], [], []
for code, data in mitre_miti.items():
    miti_codes.append(code)
    miti_names.append(data['name'])
    miti_descs.append(data['desc'])
    
MITI_SCORE_THRE = 0.1
cossim = cwe_mitre_miti_cossim(cwe_miti_curtech, miti_descs)
sort_idx = np.argsort(cossim)[::-1]
for idx in sort_idx:
    if cossim[idx] >= MITI_SCORE_THRE:
        print(miti_codes[idx], cossim[idx], miti_names[idx])
        
        
miti2def = json.load(open('/home/zxx5113/IBM/data/cyberkg_IBM/save/mitre-defend/miti2def.json', 'r'))
mitre_def = json.load(open('/home/zxx5113/IBM/data/cyberkg_IBM/save/mitre-defend/defence.json', 'r'))
def2score = defaultdict(float)
def2miti = defaultdict(set)

for idx in sort_idx:
    if cossim[idx] >= MITI_SCORE_THRE:
        miti_code = miti_codes[idx]
        for def_code in miti2def[miti_code]:
            def2score[def_code] += cossim[idx]
            def2miti[def_code].add(miti_code)
print('\n')
for def_code, def_score in sorted(def2score.items(), key=lambda item: item[1], reverse=True):
    print(def_code, round(def_score, 4),  list(def2miti[def_code]), mitre_def[def_code]['name'])
        

M1026 0.1777 Privileged Account Management
M1018 0.1617 User Account Management
M1029 0.1526 Remote Data Storage
M1022 0.1376 Restrict File and Directory Permissions
M1039 0.1276 Environment Variable Permissions
M1013 0.1118 Application Developer Guidance
M1025 0.1081 Privileged Process Integrity
M1056 0.1018 Pre-compromise


D3-LFP 0.2993 ['M1022', 'M1018'] Local File Permissions
D3-MAC 0.2698 ['M1025', 'M1018'] Mandatory Access Control
D3-DAM 0.1777 ['M1026'] Domain Account Monitoring
D3-LAM 0.1777 ['M1026'] Local Account Monitoring
D3-SPP 0.1777 ['M1026'] Strong Password Policy
D3-SCP 0.1617 ['M1018'] System Configuration Permissions
D3-ACH 0.1276 ['M1039'] Application Configuration Hardening
D3-SFA 0.1276 ['M1039'] System File Analysis
D3-BA 0.1081 ['M1025'] Bootloader Authentication
D3-DLIC 0.1081 ['M1025'] Driver Load Integrity Checking
D3-PSEP 0.1081 ['M1025'] Process Segment Execution Prevention
D3-DE 0.1018 ['M1056'] Decoy Environment
D3-DO 0.1018 ['M1056'] Decoy Object


### CWE - DEF

In [4]:
import json
import numpy as np
from collections import defaultdict
from data.cyberkg_IBM.pkg.sim_mitre import cwe_mitre_def_cossim

mitre_def = json.load(open('/home/zxx5113/IBM/data/cyberkg_IBM/save/mitre-defend/defence.json', 'r'))

def_codes, def_names, def_descs = [], [], []
for code, data in mitre_def.items():
    def_codes.append(code)
    def_names.append(data['name'])
    def_descs.append(data['desc'])
    
MITI_SCORE_THRE = 0.1
cossim = cwe_mitre_def_cossim(cwe_miti_curtech, def_descs)
sort_idx = np.argsort(cossim)[::-1]
for idx in sort_idx:
    if cossim[idx] >= MITI_SCORE_THRE:
        print(def_codes[idx], cossim[idx], def_names[idx])

D3-FE 0.1968 File Encryption
D3-ACH 0.1683 Application Configuration Hardening
D3-EAL 0.1507 Executable Allowlisting
D3-MENCR 0.1315 Message Encryption
D3-AH 0.128 Application Hardening
D3-SCF 0.1136 System Call Filtering
D3-SFA 0.1032 System File Analysis


In [ ]:
# evaluating the precision & recall
# step1: use TTP to link to the MITRE mitigation, then link to MITRE defence, use those info as ground-truth
# step2: use two approaches (TTP-CWE-MITI-DEF and TTP-CWE-DEF) to get the top-K defence list
# step3: calculate precision, recall (etc.) of top-K defence list in step2